# Macro Impact Tracker – Macro Events Data Layer

This notebook builds the macroeconomic events dataset for:
- 🇺🇸 United States (via FRED API)
- 🇮🇳 India (via official government CSV data)

The output of this notebook is a standardized macro-events table
that will later be aligned with market price data for event-based analysis.

Author: Rushank Talwar  
Goal: FinTech + Product Management portfolio project


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta


In [ ]:
# FRED API key
API_KEY = "ae5a62f0a9711d47a3657823f35a83b8"

# US macro series we care about
US_MACRO_SERIES = {
    "CPI": "CPIAUCSL",      # Consumer Price Index
    "Unemployment Rate": "UNRATE",
    "Fed Funds Rate": "FEDFUNDS"
}

START_DATE = "2019-01-01"
END_DATE = "2025-12-31"


In [ ]:
def fetch_fred_series(series_id, start_date, end_date):
    """
    Fetch time-series data from FRED
    """
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": API_KEY,
        "file_type": "json",
        "observation_start": start_date,
        "observation_end": end_date
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    data = response.json()["observations"]
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    
    return df[["date", "value"]]


In [ ]:
us_events = []

for event_name, series_id in US_MACRO_SERIES.items():
    df = fetch_fred_series(series_id, START_DATE, END_DATE)
    
    # Approximate release time (FRED does not provide intraday timestamps)
    df["datetime"] = df["date"] + pd.Timedelta(hours=8, minutes=30)
    
    df["country"] = "US"
    df["event"] = event_name
    df.rename(columns={"value": "actual"}, inplace=True)
    
    us_events.append(df[["country", "event", "datetime", "actual"]])

us_macro_df = pd.concat(us_events, ignore_index=True)
us_macro_df.head(10)


In [ ]:
india_cpi = pd.read_csv("../data/india_cpi.csv")
india_cpi.head()


In [ ]:
# Always restart from raw data
india_cpi_raw = pd.read_csv("../data/india_cpi.csv")

india_cpi_raw.head()
india_cpi_raw.columns


In [ ]:
india_cpi_raw = pd.read_csv("../data/india_cpi.csv")


In [ ]:
india_cpi_all = india_cpi_raw[
    india_cpi_raw["Sector"] == "Rural+Urban"
].copy()

india_cpi_all.head()


In [ ]:
india_cpi_all = india_cpi_all[
    ["Year", "Month", "General index"]
].rename(columns={"General index": "actual"})

india_cpi_all.head()


In [ ]:
india_cpi_all["date"] = pd.to_datetime(
    india_cpi_all["Year"].astype(str) + "-" +
    india_cpi_all["Month"].astype(str) + "-01",
    errors="coerce"
)


In [ ]:
india_cpi_all["actual"] = pd.to_numeric(
    india_cpi_all["actual"], errors="coerce"
)

india_cpi_all = india_cpi_all.dropna(subset=["date", "actual"])
india_cpi_all = india_cpi_all[india_cpi_all["date"] >= "2016-01-01"]


In [ ]:
india_cpi_all["country"] = "INDIA"
india_cpi_all["event"] = "India CPI (Rural+Urban)"

# CPI release ~5 PM IST
india_cpi_all["datetime"] = india_cpi_all["date"] + pd.Timedelta(hours=17)

india_cpi_final = india_cpi_all[
    ["country", "event", "date", "datetime", "actual"]
].sort_values("date")

india_cpi_final.head()


In [ ]:
india_repo = pd.read_csv("../data/india_reporate.csv")
india_repo.head(10)


In [ ]:
import pandas as pd

# Reload raw repo data
india_repo_raw = pd.read_csv("../data/india_reporate.csv")

# Rename columns for sanity
india_repo_raw.columns = ["raw_date", "actual"]

# Drop fully empty rows
india_repo_raw = india_repo_raw.dropna(how="all")

# Identify year rows (e.g., 2025, 2024, etc.)
india_repo_raw["year"] = pd.to_numeric(india_repo_raw["raw_date"], errors="coerce")

# Forward-fill year values
india_repo_raw["year"] = india_repo_raw["year"].ffill()

# Keep only rows where actual repo rate exists
india_repo_clean = india_repo_raw[
    pd.to_numeric(india_repo_raw["actual"], errors="coerce").notna()
].copy()

# Convert repo rate to numeric
india_repo_clean["actual"] = pd.to_numeric(
    india_repo_clean["actual"], errors="coerce"
)

# Build date from Year + Month
india_repo_clean["date"] = pd.to_datetime(
    india_repo_clean["year"].astype(int).astype(str) + "-" +
    india_repo_clean["raw_date"].astype(str) + "-01",
    errors="coerce"
)

# Drop bad rows
india_repo_clean = india_repo_clean.dropna(subset=["date", "actual"])

# Keep MPC regime only (post Feb 2016)
india_repo_clean = india_repo_clean[
    india_repo_clean["date"] >= "2016-02-01"
]

# Add metadata
india_repo_clean["country"] = "INDIA"
india_repo_clean["event"] = "RBI Policy Repo Rate"

# RBI policy announcements ~10 AM IST
india_repo_clean["datetime"] = (
    india_repo_clean["date"] + pd.Timedelta(hours=10)
)

# Final schema
india_repo_final = india_repo_clean[
    ["country", "event", "date", "datetime", "actual"]
].sort_values("date")

india_repo_final.head(80)


In [ ]:
# Combine India macro events
india_macro_df = pd.concat(
    [india_cpi_final, india_repo_final],
    ignore_index=True
)

# Combine with US macro (already created earlier)
macro_events_all = pd.concat(
    [us_macro_df, india_macro_df],
    ignore_index=True
)

macro_events_all = macro_events_all.sort_values("datetime")
macro_events_all.head(100)


In [ ]:
macro_events_all.info()
macro_events_all.isna().sum()


In [ ]:
# 1. Drop rows with missing actual values
macro_events_all = macro_events_all.dropna(subset=["actual"])

# 2. Rebuild 'date' consistently from datetime
macro_events_all["date"] = macro_events_all["datetime"].dt.date
macro_events_all["date"] = pd.to_datetime(macro_events_all["date"])

# 3. Final sanity check
macro_events_all.info()
macro_events_all.isna().sum()


In [ ]:
macro_events_all.to_csv("../data/macro_events_all.csv", index=False)
print("Clean macro_events_all.csv saved")
